# **Комп’ютерний практикум №5. Навчання рекурентних нейронних мереж засобами TensorFlow**

In [ ]:
import librosa
import numpy as np
import pandas as pd
import os

import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras import layers 
from tensorflow.keras.preprocessing import timeseries_dataset_from_array
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D,Dropout,Activation
from sklearn.model_selection import train_test_split
from keras.layers import TimeDistributed,GRU
import re
import matplotlib.pyplot as plt
from IPython import display
from IPython.display import clear_output
import glob
import imageio
import time
import IPython.display as ipd
import sys

AUTOTUNE = tf.data.experimental.AUTOTUNE

seed=123

# Допоміжні функції

In [ ]:
def results_plot(history):
    
    plt.figure(figsize=(17,5))
    plt.plot(history.history['loss'], color='b', label="train loss")
    plt.plot(history.history['val_loss'], color='r', label="val loss")

    plt.plot()
    
def plot_preds(y_test, preds):
    
    plt.figure(figsize=(17,5))
    plt.plot(preds, label='preds', color="g",alpha=0.7)
    plt.plot(y_test, label='test',alpha=0.7,color="yellow")
    plt.legend()
    plt.title('Real test and predicted')
    plt.show()

In [ ]:
def mae(y_true, y_pred):
    output_errors = np.average(np.abs(y_pred - y_true), axis=0)
    return np.average(output_errors)

def mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def rmse(y_true, y_pred):
    return np.sqrt(((y_pred - y_true) ** 2).mean())

def brief_stats(y_true, y_pred):
    print(pd.Series({
        'mape': mape(y_true, y_pred),
        'mae': mae(y_true, y_pred),
        'rmse': rmse(y_true, y_pred),
    }))

# Завантаження мелодії

In [ ]:
def load(file_):
    data_, sampling_rate = librosa.load(file_, offset=0.0, duration=138)
    print(sampling_rate)
    data_ = data_.reshape(1,3042900)
    return data_
map_data = lambda filename: tf.compat.v1.py_func(load, [filename], [tf.float32])

In [ ]:
track=load("../input/songsdata/v-peshhere-gornogo-korolja.wav")

# Підготуємо мелодію до навчання

In [ ]:
track.shape

In [ ]:
track=track[:,1700000:2202900]

In [ ]:
ipd.Audio(track,rate=22050)

In [ ]:
import librosa.display
plt.figure(figsize=(9,7))
librosa.display.waveplot(track[0], sr=22050)

In [ ]:
def get_chunks(train, n_input, n_out=10):
    X, y = list(), list()
    in_start = 0
    for _ in range(len(train)):
        in_end = in_start + n_input
        out_end = in_end + n_out
        if out_end <= len(train):
            x_input = train[in_start:in_end]
            x_input = x_input.reshape((len(x_input), 1))
            X.append(x_input)
            y.append(train[in_end:out_end])
            in_start += 1
    return np.array(X),np.array(y)

In [ ]:
track = track.reshape(-1, 1).flatten()
train, test = train_test_split(track, test_size=0.5, shuffle=False)

In [ ]:
track.shape

In [ ]:
n_input = 151000
X_train, y_train = get_chunks(train, n_input=n_input, n_out=100000)
X_test, y_test = get_chunks(test, n_input=n_input, n_out=100000)

In [ ]:
X_train.shape

In [ ]:
X_test.shape[0]

In [ ]:
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [ ]:
fit_params = {
        'x': X_train,
        'y': y_train,
        'validation_data': (X_train, y_train),
        'verbose': 1,
        'epochs': 20,
        'batch_size': 200}

# SimpleRNN model

In [ ]:
from keras.layers import SimpleRNN

In [ ]:
def base_rnn():
    model = Sequential()
    model.add(SimpleRNN(1, input_shape=(1,n_input),
                   return_sequences=False
                  ))
    model.compile(loss='mse', optimizer='adam')
    return model

In [ ]:
rnn_model = base_rnn()
history_rnn =  rnn_model.fit(**fit_params)
results_plot(history_rnn)

preds_train = rnn_model.predict(X_train).flatten()
plot_preds(y_train[:, 0], preds_train)
brief_stats(y_train[:, 0], preds_train)

preds_test = rnn_model.predict(X_test).flatten()
plot_preds(y_test[:, 0], preds_test)
brief_stats(y_test[:, 0], preds_test)

plot_preds(y_train[10:30, 0], preds_train[10:30])

plot_preds(y_test[10:30, 0], preds_test[10:30])

In [ ]:
#preds.shape

In [ ]:
#preds.reshape(1,200573)

In [ ]:
#ipd.Audio(preds,rate=23050)

In [ ]:
#ipd.Audio(track[:][-200573:],rate=23050)

# simple LTSM model

In [ ]:
def lstm1():
    model = Sequential()
    model.add(LSTM(100, activation='relu', input_shape=(1,n_input),
                   return_sequences=True
                  ))
    model.add(TimeDistributed(Dense(1)))
    model.compile(loss='mse', optimizer='adam')
    return model

In [ ]:
lstm_model = lstm1()
history_lstm =  lstm_model.fit(**fit_params)
results_plot(history_lstm)

preds_train = lstm_model.predict(X_train).flatten()
plot_preds(y_train[:, 0], preds_train)
brief_stats(y_train[:, 0], preds_train)

preds_test = lstm_model.predict(X_test).flatten()
plot_preds(y_test[:, 0], preds_test)
brief_stats(y_test[:, 0], preds_test)

plot_preds(y_train[10:30, 0], preds_train[10:30])

plot_preds(y_test[10:30, 0], preds_test[10:30])

# simple GRU model

In [ ]:
def gru1():
    model = Sequential()
    model.add(GRU(100, activation='relu', input_shape=(1,n_input),
                   return_sequences=True
                  ))
    model.add(TimeDistributed(Dense(1)))
    model.compile(loss='mse', optimizer='adam')
    return model

In [ ]:
gru_model = gru1()
history_gru = gru_model.fit(**fit_params)
results_plot(history_gru)

preds_train = gru_model.predict(X_train).flatten()
plot_preds(y_train[:, 0], preds_train)
brief_stats(y_train[:, 0], preds_train)

preds_test = gru_model.predict(X_test).flatten()
plot_preds(y_test[:, 0], preds_test)
brief_stats(y_test[:, 0], preds_test)

plot_preds(y_train[10:30, 0], preds_train[10:30])

plot_preds(y_test[10:30, 0], preds_test[10:30])

# Попудуємо більш складні моделі

In [ ]:
def lstm2():
    model = Sequential()
    model.add(LSTM(100, activation='relu', input_shape=(1,n_input),
                   return_sequences=True,
                   recurrent_dropout = 0.1
                  ))

    model.add(LSTM(20, activation='relu', input_shape=(1,n_input),
                   return_sequences=True,
                   recurrent_dropout = 0.1
                  ))
    model.add(TimeDistributed(Dense(9)))
    model.add(TimeDistributed(Dense(1)))
    model.compile(loss='mse', optimizer='adam')
    return model

In [ ]:
lstm_model = lstm2()
history_lstm =  lstm_model.fit(**fit_params)
results_plot(history_lstm)

preds_train = lstm_model.predict(X_train).flatten()
plot_preds(y_train[:, 0], preds_train)
brief_stats(y_train[:, 0], preds_train)

preds_test = lstm_model.predict(X_test).flatten()
plot_preds(y_test[:, 0], preds_test)
brief_stats(y_test[:, 0], preds_test)

plot_preds(y_train[10:30, 0], preds_train[10:30])

plot_preds(y_test[10:30, 0], preds_test[10:30])

In [ ]:
def gru2():
    model = Sequential()
    model.add(GRU(100, activation='relu', input_shape=(1,n_input),
                   return_sequences=True,
                   recurrent_dropout = 0.1
                  ))
    model.add(GRU(40, activation='relu', input_shape=(1,n_input),
                   return_sequences=True,
                   recurrent_dropout = 0.1
                  ))
    model.add(TimeDistributed(Dense(9)))
    model.add(TimeDistributed(Dense(1)))
    model.compile(loss='mse', optimizer='adam')
    return model

In [ ]:
gru_model = gru2()
history_gru = gru_model.fit(**fit_params)
results_plot(history_gru)

preds_train = gru_model.predict(X_train).flatten()
plot_preds(y_train[:, 0], preds_train)
brief_stats(y_train[:, 0], preds_train)

preds_test = gru_model.predict(X_test).flatten()
plot_preds(y_test[:, 0], preds_test)
brief_stats(y_test[:, 0], preds_test)

plot_preds(y_train[10:30, 0], preds_train[10:30])

plot_preds(y_test[10:30, 0], preds_test[10:30])

In [ ]:
def lstm3():
    model = Sequential()
    model.add(LSTM(100, input_shape=(1,n_input),
                   return_sequences=True,
                   dropout = 0.2
                  ))
    model.add(LSTM(40,input_shape=(1,n_input),
                   return_sequences=True,
                  ))
    model.add(TimeDistributed(Dense(32)))
    model.add(TimeDistributed(Dense(1)))
    model.compile(loss='mse', optimizer='adam')
    return model

In [ ]:
lstm_model = lstm3()
history_lstm =  lstm_model.fit(**fit_params)
results_plot(history_lstm)

preds_train = lstm_model.predict(X_train).flatten()
plot_preds(y_train[:, 0], preds_train)
brief_stats(y_train[:, 0], preds_train)

preds_test = lstm_model.predict(X_test).flatten()
plot_preds(y_test[:, 0], preds_test)
brief_stats(y_test[:, 0], preds_test)

plot_preds(y_train[10:30, 0], preds_train[10:30])

plot_preds(y_test[10:30, 0], preds_test[10:30])

In [ ]:
def gru3():
    model = Sequential()
    model.add(GRU(200, input_shape=(1,n_input),
                   return_sequences=True,
                   dropout = 0.05
                  ))
    model.add(Dropout(0.3))
    model.add(GRU(100,input_shape=(1,n_input),
                   return_sequences=True,
                  ))
    model.add(TimeDistributed(Dense(32)))
    model.add(TimeDistributed(Dense(1)))
    model.compile(loss='mse', optimizer='adam')
    return model

In [ ]:
gru_model = gru3()
history_gru = gru_model.fit(**fit_params)
results_plot(history_gru)

preds_train = gru_model.predict(X_train).flatten()
plot_preds(y_train[:, 0], preds_train)
brief_stats(y_train[:, 0], preds_train)

preds_test = gru_model.predict(X_test).flatten()
plot_preds(y_test[:, 0], preds_test)
brief_stats(y_test[:, 0], preds_test)

plot_preds(y_train[10:30, 0], preds_train[10:30])

plot_preds(y_test[10:30, 0], preds_test[10:30])

#  Глибокі моделі зi згортковими шарами conv1D та шарами max_pool1D

In [ ]:
from keras.layers import Conv1D, MaxPooling1D, Conv2D, Flatten

def conv_gru():
    model = Sequential()
    model.add(Conv1D(filters=20, kernel_size=2, activation='relu',
                     input_shape=(X_test_1d.shape[1],X_test_1d.shape[2])))
    model.add(MaxPooling1D())
    model.add(GRU(100, activation='relu', 
                   return_sequences=True,
                  ))
    model.add(Flatten())

    model.add(Dense(20))
    model.add(Dropout(0.3))
    model.add(Dense(1))
    model.compile(loss='mse', optimizer='adam')
    return model

In [ ]:
def conv_lstm():
    model = Sequential()
    model.add(Conv1D(filters=20, kernel_size=2, activation='relu',
                     input_shape=(X_test_1d.shape[1],X_test_1d.shape[2])))
    model.add(MaxPooling1D())

    model.add(LSTM(100, activation='relu', 
                   return_sequences=True,
                  ))
    model.add(Flatten())

    model.add(Dense(20))
    model.add(Dropout(0.3))
    model.add(Dense(1))
    model.compile(loss='mse', optimizer='adam')
    return model

In [ ]:
X_train_1d = X_train.reshape(X_train.shape[0], X_train.shape[2], X_train.shape[1],)
X_test_1d = X_test.reshape(X_test.shape[0], X_test.shape[2], X_test.shape[1],)

In [ ]:
conv_lstm_model = conv_lstm()
history = conv_lstm_model.fit(X_train_1d, y_train,
                                   validation_data = (X_test_1d, y_test),
                                   batch_size =50,
                                   epochs = 20)
results_plot(history)

In [ ]:
preds_train = conv_lstm_model.predict(X_train_1d).flatten()
plot_preds(y_train[:, 0], preds_train)
brief_stats(y_train[:, 0], preds_train)

preds_test = conv_lstm_model.predict(X_test_1d).flatten()
plot_preds(y_test[:, 0], preds_test)
brief_stats(y_test[:, 0], preds_test)

plot_preds(y_train[10:30, 0], preds_train[10:30])

plot_preds(y_test[10:30, 0], preds_test[10:30])

In [ ]:
conv_gru_model = conv_gru()
history = conv_gru_model.fit(X_train_1d, y_train,
                                   validation_data = (X_test_1d, y_test),
                                   batch_size =50,
                                   epochs = 10)
results_plot(history)

preds_train = conv_gru_model.predict(X_train_1d).flatten()
plot_preds(y_train[:, 0], preds_train)
brief_stats(y_train[:, 0], preds_train)

preds_test = conv_gru_model.predict(X_test_1d).flatten()
plot_preds(y_test[:, 0], preds_test)
brief_stats(y_test[:, 0], preds_test)

plot_preds(y_train[10:30, 0], preds_train[10:30])

plot_preds(y_test[10:30, 0], preds_test[10:30])

In [ ]:
from statsmodels.graphics.tsaplots import plot_pacf, plot_acf
plot_acf(track)
plot_pacf(track)

# Генерація мелодії за дпомогою найкращої моделі

In [ ]:
fit_params = {
        'x': X_train,
        'y': y_train,
        'validation_data': (X_train, y_train),
        'verbose': 1,
        'epochs': 10,
        'batch_size': 10}

In [ ]:
def gru():
    model = Sequential()
    model.add(GRU(500, activation='relu', input_shape=(1,n_input),
                   return_sequences=True,
                   recurrent_dropout = 0.2
                  ))
    model.add(GRU(500, activation='relu', input_shape=(1,n_input),
                   return_sequences=True,
                   recurrent_dropout = 0.2
                  ))
    model.add(TimeDistributed(Dense(100000)))
    model.compile(loss='mse', optimizer='adam')
    return model

In [ ]:
gru_model = gru()
history_gru = gru_model.fit(**fit_params)
results_plot(history_gru)

In [ ]:
X_test[-2:-1]

In [ ]:
preds = gru_model.predict(X_test[-2:-1]).flatten()

In [ ]:
preds, y_test[-1]

In [ ]:
np.array(preds).shape

In [ ]:
preds=np.array(preds).reshape(1,100000)
y=np.array(y_test[-1]).reshape(1,100000)

In [ ]:
ipd.Audio(preds,rate=22050)

In [ ]:
ipd.Audio(y,rate=22050)

In [ ]:
import librosa.display
plt.figure(figsize=(9,7))
librosa.display.waveplot(preds[0], sr=22050)

In [ ]:
plt.figure(figsize=(9,7))
librosa.display.waveplot(y[0], sr=22050)